In [ ]:
%pip install ipython-sql
%pip install tabulate
%pip install -q pandas

In [2]:
import csv, sqlite3

con = sqlite3.connect("ChicagoDataBase.db")
cur = con.cursor()

In [3]:
import pandas

csv_names   = ['ChicagoCensusData.csv', 'ChicagoCrimeData.csv', 'ChicagoPublicSchools.csv']
table_names = ['censusData',            'crimeData',            'publicSchools']

for csv_name, db_name in zip(csv_names, table_names):
    df = pandas.read_csv(csv_name)
    df.to_sql(db_name, con, if_exists='replace', index=False)

In [5]:
from tabulate import tabulate

Problem 1: Find the total number of crimes recorded in the CRIME table.

In [6]:
cur.execute("""
    SELECT
        COUNT(DISTINCT ID)
    FROM
        crimeData
""")

print(
    tabulate(
        cur.fetchall(),
        headers= [
            'COUNT(DISTINCT ID)'
        ]
    )
)

  COUNT(DISTINCT ID)
--------------------
                 533


Problem 2: List community areas with per capita income less than 11000.

In [7]:
cur.execute("""
    SELECT
        COMMUNITY_AREA_NUMBER,
        COMMUNITY_AREA_NAME,
        PER_CAPITA_INCOME
    FROM
        censusData
    WHERE
        PER_CAPITA_INCOME < 11000
""")

print(
    tabulate(
        cur.fetchall(),
        headers= [
            'COMMUNITY_AREA_NUMBER',
            'COMMUNITY_AREA_NAME',
            'PER_CAPITA_INCOME'
        ]
    )
)

  COMMUNITY_AREA_NUMBER  COMMUNITY_AREA_NAME      PER_CAPITA_INCOME
-----------------------  ---------------------  -------------------
                     26  West Garfield Park                   10934
                     30  South Lawndale                       10402
                     37  Fuller Park                          10432
                     54  Riverdale                             8201


Problem 3: List all case numbers for crimes  involving minors?(children are not considered minors for the purposes of crime analysis)

In [8]:
cur.execute("""
    SELECT
        ID,
        CASE_NUMBER,
        PRIMARY_TYPE,
        DESCRIPTION
    FROM
        crimeData
    WHERE
        LOWER(DESCRIPTION) LIKE '%minor%'
""")

print(
    tabulate(
        cur.fetchall(),
        headers= [
            'ID',
            'CASE_NUMBER',
            'PRIMARY_TYPE',
            'DESCRIPTION'
        ]
    )
)

     ID  CASE_NUMBER    PRIMARY_TYPE          DESCRIPTION
-------  -------------  --------------------  -----------------------------
3987219  HL266884       LIQUOR LAW VIOLATION  SELL/GIVE/DEL LIQUOR TO MINOR
3266814  HK238408       LIQUOR LAW VIOLATION  ILLEGAL CONSUMPTION BY MINOR


Problem 4: List all kidnapping crimes involving a child?

In [9]:
cur.execute("""
    SELECT
        ID,
        CASE_NUMBER,
        PRIMARY_TYPE,
        DESCRIPTION
    FROM
        crimeData
    WHERE
        PRIMARY_TYPE = 'KIDNAPPING'
        AND LOWER(DESCRIPTION) LIKE '%child%'
""")

print(
    tabulate(
        cur.fetchall(),
        headers= [
            'ID',
            'CASE_NUMBER',
            'PRIMARY_TYPE',
            'DESCRIPTION'
        ]
    )
)

     ID  CASE_NUMBER    PRIMARY_TYPE    DESCRIPTION
-------  -------------  --------------  ------------------------
5276766  HN144152       KIDNAPPING      CHILD ABDUCTION/STRANGER


Problem 5: What kind of crimes were recorded at schools?

In [10]:
cur.execute("""
    SELECT
        ID,
        CASE_NUMBER,
        PRIMARY_TYPE,
        DESCRIPTION,
        LOCATION_DESCRIPTION
    FROM
        crimeData
    WHERE
        LOWER(LOCATION_DESCRIPTION) LIKE '%school%'
""")

print(
    tabulate(
        cur.fetchall(),
        headers= [
            'ID',
            'CASE_NUMBER',
            'PRIMARY_TYPE',
            'DESCRIPTION',
            'LOCATION_DESCRIPTION'
        ]
    )
)

      ID  CASE_NUMBER    PRIMARY_TYPE            DESCRIPTION                     LOCATION_DESCRIPTION
--------  -------------  ----------------------  ------------------------------  -------------------------
 4006321  HL353697       BATTERY                 SIMPLE                          SCHOOL, PUBLIC, GROUNDS
 4430638  HL725506       BATTERY                 PRO EMP HANDS NO/MIN INJURY     SCHOOL, PUBLIC, BUILDING
 6644618  HP716225       BATTERY                 SIMPLE                          SCHOOL, PUBLIC, BUILDING
 2341955  HH639427       BATTERY                 SIMPLE                          SCHOOL, PUBLIC, BUILDING
11110571  JA460432       BATTERY                 SIMPLE                          SCHOOL, PUBLIC, GROUNDS
 7399281  HS200939       CRIMINAL DAMAGE         TO VEHICLE                      SCHOOL, PUBLIC, GROUNDS
 3530721  HK577020       NARCOTICS               POSS: HEROIN(WHITE)             SCHOOL, PUBLIC, GROUNDS
 7502426  HS305355       NARCOTICS               MANU

Problem 6: List the average safety score for all types of schools.

In [11]:
cur.execute("""
    SELECT
        "Elementary, Middle, or High School",
        AVG(SAFETY_SCORE)
    FROM
        publicSchools
    GROUP BY
        "Elementary, Middle, or High School"
""")

print(
    tabulate(
        cur.fetchall(),
        headers= [
            "Elementary, Middle, or High School",
            'AVG(SAFETY_SCORE)'
        ]
    )
)

Elementary, Middle, or High School      AVG(SAFETY_SCORE)
------------------------------------  -------------------
ES                                                49.5204
HS                                                49.6235
MS                                                48


Problem 7: List 5 community areas with highest % of households below poverty line.

In [12]:
cur.execute("""
    SELECT
        COMMUNITY_AREA_NUMBER,
        COMMUNITY_AREA_NAME,
        PERCENT_HOUSEHOLDS_BELOW_POVERTY
    FROM
        censusData
    ORDER BY
        PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC
    LIMIT 5
""")

print(
    tabulate(
        cur.fetchall(),
        headers= [
            'COMMUNITY_AREA_NUMBER',
            'COMMUNITY_AREA_NAME',
            'PERCENT_HOUSEHOLDS_BELOW_POVERTY'
        ]
    )
)

  COMMUNITY_AREA_NUMBER  COMMUNITY_AREA_NAME      PERCENT_HOUSEHOLDS_BELOW_POVERTY
-----------------------  ---------------------  ----------------------------------
                     54  Riverdale                                            56.5
                     37  Fuller Park                                          51.2
                     68  Englewood                                            46.6
                     29  North Lawndale                                       43.1
                     27  East Garfield Park                                   42.4


Problem 8: Which community area(number) is most crime prone?

In [13]:
cur.execute("""
    SELECT
        COMMUNITY_AREA_NUMBER,
        COUNT(*) AS 'numberOfCrimes'
    FROM
        crimeData
    GROUP BY
        COMMUNITY_AREA_NUMBER
    ORDER BY
        numberOfCrimes DESC
    LIMIT 1
""")

print(
    tabulate(
        cur.fetchall(),
        headers= [
            'COMMUNITY_AREA_NUMBER',
            'numberOfCrimes'
        ]
    )
)

  COMMUNITY_AREA_NUMBER    numberOfCrimes
-----------------------  ----------------
                     25                43


Problem 9: Use a sub-query to find the name of the community area with highest hardship index.

In [14]:
cur.execute("""
    SELECT
        COMMUNITY_AREA_NUMBER,
        COMMUNITY_AREA_NAME,
        HARDSHIP_INDEX
    FROM
        censusData
    WHERE
        HARDSHIP_INDEX = (
            SELECT
                MAX(HARDSHIP_INDEX)
            FROM
                censusData
        )
""")

print(
    tabulate(
        cur.fetchall(),
        headers= [
            'COMMUNITY_AREA_NUMBER',
            'COMMUNITY_AREA_NAME',
            'HARDSHIP_INDEX'
        ]
    )
)

  COMMUNITY_AREA_NUMBER  COMMUNITY_AREA_NAME      HARDSHIP_INDEX
-----------------------  ---------------------  ----------------
                     54  Riverdale                            98


Problem 10: Use a sub-query to determine the Community Area Name with most number of crimes?

In [15]:
cur.execute("""
    SELECT
        COMMUNITY_AREA_NAME
    FROM
        censusData
    WHERE
        COMMUNITY_AREA_NUMBER = (
            SELECT
                COMMUNITY_AREA_NUMBER
            FROM
                censusData
            GROUP BY
                COMMUNITY_AREA_NUMBER
            ORDER BY
                COUNT(COMMUNITY_AREA_NUMBER) DESC
            LIMIT 1
        )
    LIMIT 1
""")

print(
    tabulate(
        cur.fetchall(),
        headers= [
            'COMMUNITY_AREA_NAME'
        ]
    )
)

COMMUNITY_AREA_NAME
---------------------
Edgewater
